In [1]:
# Environment variables
drive = False
install_packages = False

# If necessary, install packages
if install_packages:
  !pip install Basemap basemap-data-hires
  !pip install pandas matplotlib numpy
  !pip install sklearn
  !pip install datetime

# Imports
import os
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from csv import reader
from sklearn.metrics import r2_score
from datetime import datetime
from datetime import timedelta as tdelta
from datetime import time as time
import warnings
import osmnx as ox
import folium
import networkx as nx
from matplotlib import colors

warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.simplefilter(action='ignore', category=pd.errors.PerformanceWarning)
warnings.simplefilter(action='ignore', category=pd.errors.SettingWithCopyWarning)

# Mount drive if needed
if drive:
  from google.colab import drive
  drive.mount('/content/drive')
  %cd /content/drive/MyDrive/LSIS

from Helper_funcs import *
from SensorPositions_Func import *
from SVD_Func import *

# Depending on the environemnt, set directory to data
if drive:
  dir_path = '/content/drive/MyDrive/LSIS/data/'
else :
  dir_path = './data/'

folders = ['region_1_mustamäe_kristiine', 'region_2_data_kesklinn', 'region_3_kadriorg_lasnamäe', 'region_4_ülemiste']
columns_sensors_positions = ['IDs', 'latitude', 'longitude', 'region']

# 1) Import Data #

In [2]:
csv_file_name = 'sensor_positions.csv'
coords = []
IDs = []

with open(dir_path + csv_file_name, 'r') as read_obj:
    csv_reader = reader(read_obj)
    for row in csv_reader:
        coord = (float(row[0].split(' ')[0].replace('(', '')), 
                 float(row[0].split(' ')[1].replace(')', '')) )
        coords.append(coord)
        
        IDs.append(row[1])

stations_df = import_sensor_positions(dir_path, csv_file_name)
stations_df.set_index('IDs', inplace=True)
IDs = list(stations_df.index)

del csv_file_name, coords, csv_reader, read_obj, row

In [3]:
df_data_interp = pd.read_csv(dir_path + 'interpolated_data_1.csv');
tindex = pd.to_datetime(df_data_interp[df_data_interp.columns[1]]);
df_data_interp.set_index(tindex, inplace=True);
df_data_interp.drop(['Unnamed: 0.1', 'Unnamed: 0'], axis = 1, inplace=True);

df_day = df_data_interp.iloc[indexer_day(df_data_interp)]
df_eve = df_data_interp.iloc[indexer_evening(df_data_interp)]
df_night = df_data_interp.iloc[indexer_night(df_data_interp)]

stations_df['m_day'] = df_day.mean()
stations_df['m_eve'] = df_eve.mean()
stations_df['m_nig'] = df_night.mean()



del df_data_interp, df_day, df_eve, df_night


# 2) Download maps #

In [4]:
#from shapely.geometry import Polygon
#poly = [
#    (59.376750, 24.717667),
#    (59.378149, 24.638082),
#    (59.407865, 24.622976),
#    (59.452295, 24.732891),
#    (59.451440, 24.859805),
#    (59.436058, 24.874094)
#    ]
lower_left = (59.365370, 24.627558)
upper_right = (59.458556, 24.882702)

graph = ox.graph_from_bbox(upper_right[0], lower_left[0], upper_right[1], lower_left[1], network_type='drive')
#polyg = Polygon(poly)
#graph = ox.graph_from_polygon(polyg, network_type='drive')

data = ox.graph_to_gdfs(graph, nodes=False)

# Calculate the center of the map
center = ((lower_left[0] + upper_right[0]) / 2, (lower_left[1] + upper_right[1]) / 2)

# Create a folium map centered around the calculated center


# 3) Assign sensors to streets + erase redundant #

In [5]:
tdata = [[] for _ in range(len(stations_df.index))]
stations_df['nearest_ed']= tdata
stations_df['nearest_ed1']= tdata
nearest_edges = []

for sid in stations_df.index:
    stations_df['nearest_ed'][sid] = ox.distance.nearest_edges(graph, X=stations_df['latitude'][sid], Y=stations_df['longitude'][sid])
    
    nearest_edges.append(stations_df['nearest_ed'][sid])

graph_temp = graph.copy()

for sid in stations_df.index:
    graph_temp.remove_edge(stations_df['nearest_ed'][sid][0], stations_df['nearest_ed'][sid][1])
    stations_df['nearest_ed1'][sid] = ox.distance.nearest_edges(graph_temp, X=stations_df['latitude'][sid], Y=stations_df['longitude'][sid])
    graph_temp = graph.copy()
    nearest_edges.append(stations_df['nearest_ed1'][sid])
    
nearest_edges = pd.DataFrame(data={'edges':nearest_edges})
nearest_edges.drop_duplicates(subset='edges', keep='first', inplace=True)
    


del tdata, graph_temp

In [6]:
nearest_edges.set_index(nearest_edges['edges'], inplace=True)

2) Find some neighbors: 
2.1) Go through all edges, that are near a station and have a name.
2.2) Find all edges, that hame some of the names of the already found closest edges, but are not in 'nearest_edges' DataFrame

In [20]:
nearest_names = []
nearest_edges['name'] = ''

# get names of detected nearest edges
for edge in nearest_edges.index:
    if 'name' in graph.edges[edge] and graph.edges[edge]['name'] not in nearest_names:
        tname = graph.edges[edge]['name']
        nearest_names.append(tname)
        nearest_edges['name'][edge] = tname;
        
        
named_edges = []
# get all edges, that are named and other edge of the same name is in 
# nearest_edges
for edge in graph.edges:
    if 'name' in graph.edges[edge] and graph.edges[edge]['name'] in nearest_names:
        named_edges.append(edge);

# filter edges, that arent present in index already        
n_edges = list(nearest_edges.index.values);
to_add = [item for item in named_edges if item not in n_edges]

df_to_add = pd.DataFrame(columns=nearest_edges.columns)
df_to_add['edges'] = to_add;
df_to_add.set_index(df_to_add['edges'], inplace=True)
df_to_add['n_edge'] = [()]*len(df_to_add)
df_to_add['nn_edge'] = [[]]*len(df_to_add)

for edge in df_to_add.index:
    df_to_add['name'][edge] = graph.edges[edge]['name']
    
for edge in nearest_edges['edges']:
    name = nearest_edges['name'][edge]
    if type(name) == list:
        nearest_edges['name'][edge] = name[0];

for edge in df_to_add['edges']:
    name = df_to_add['name'][edge]
    if type(name) == list:
        df_to_add['name'][edge] = name[0];

del to_add, nearest_names, named_edges, n_edges


3) Go through newly found edges and find their closest edges from the original set of 'nearest edges'

In [21]:
# go throug all named streets and 
import math
from geopy.distance import geodesic

def get_close_edges(df_to_add, nearest_edges, graph):
    i = 0
    mm = len(df_to_add)
    for edge1 in df_to_add['edges']:
        i = i+1
        print(i/mm*100)
        tname = df_to_add['name'][edge1]
        idx1 = (nearest_edges['name'] == tname)
        idx2 = ((df_to_add['name'] == tname) * (df_to_add['edges'] != edge1))
        
        # exclude unwanted edges
        tnot = df_to_add['nn_edge'][edge1]
        pp = []
        if len(tnot) > 0:
            #print(edge1)
            #print(tnot)
            pp = [(df_to_add['edges'] != ed) for ed in tnot];
            for p in pp:
                #print(p)
                idx2 = idx2*p;
            
        #print(idx2.sum())
        edges = pd.concat([nearest_edges[idx1]['edges'], df_to_add[idx2]['edges']]);
        #print(len(edges))
        #edges = edges.drop(edges == edge1)
        #print(len(edges))
        total_distance =[]

        for edge2 in edges:
            edge1_start = graph.nodes[edge1[0]]['y'], graph.nodes[edge1[0]]['x']
            edge1_end = graph.nodes[edge1[1]]['y'], graph.nodes[edge1[1]]['x']
            edge2_start = graph.nodes[edge2[0]]['y'], graph.nodes[edge2[0]]['x']
            edge2_end = graph.nodes[edge2[1]]['y'], graph.nodes[edge2[1]]['x']


            distance1 = geodesic(edge1_start, edge2_start).m
            distance2 = geodesic(edge1_end, edge2_end).m
            distance3 = geodesic(edge1_start, edge2_end).m
            distance4 = geodesic(edge1_end, edge2_start).m

            # Total distance between the edges
            total_distance.append(min([distance1, distance2, distance3, distance4]))
        
        if(min(total_distance) < 1000):    
            nearest = edges[total_distance.index(min(total_distance))]
            if nearest in nearest_edges.index:
                df_to_add['n_edge'][edge1] = nearest
            
            # see if two edges don't point on each other
            elif df_to_add['n_edge'][nearest] == df_to_add['edges'][edge1]:
                # save current nearest into the unwanted
                ii = total_distance.index(min(total_distance))
                df_to_add['nn_edge'][edge1].append(nearest)
                
                # find the 2nd nearest
                total_distance.pop(ii)
                nearest = edges[total_distance.index(min(total_distance))]
                df_to_add['n_edge'][edge1] = nearest
            else:
                df_to_add['n_edge'][edge1] = nearest
        else:
            df_to_add.drop(edge1);
        
        #del distance1, distance2, distance3, distance4, nearest, #total_distance, edge1, edge2, edges, edge1_end, edge1_start, #edge2_end, edge2_start
        #del tname, idx1, idx2
        
    return df_to_add

df_to_add = get_close_edges(df_to_add, nearest_edges, graph)


0.06605019815059446
0.13210039630118892
0.19815059445178335
0.26420079260237783
0.33025099075297226
0.3963011889035667
0.4623513870541612
0.5284015852047557
0.59445178335535
0.6605019815059445
0.726552179656539
0.7926023778071334
0.8586525759577279
0.9247027741083224
0.9907529722589168
1.0568031704095113
1.1228533685601056
1.1889035667107
1.2549537648612945
1.321003963011889
1.3870541611624834
1.453104359313078
1.5191545574636725
1.5852047556142668
1.6512549537648615
1.7173051519154559
1.7833553500660502
1.8494055482166447
1.915455746367239
1.9815059445178336
2.0475561426684283
2.1136063408190227
2.179656538969617
2.2457067371202113
2.3117569352708056
2.3778071334214
2.4438573315719947
2.509907529722589
2.5759577278731833
2.642007926023778
2.7080581241743724
2.7741083223249667
2.8401585204755615
2.906208718626156
2.97225891677675
3.038309114927345
3.104359313077939
3.1704095112285335
3.2364597093791283
3.302509907529723
3.368560105680317
3.4346103038309117
3.500660501981506
3.566710700

# 4) Assign colors #

In [22]:
# Choosing a colormap
colormap = plt.colormaps.get_cmap('jet') 

# Normalizing the values between 0 and 1



nearest_edges['ddB1'] = np.zeros(len(nearest_edges))
nearest_edges['ddB2'] = np.zeros(len(nearest_edges))
nearest_edges['ddB'] = np.zeros(len(nearest_edges))
nearest_edges['edB1'] = np.zeros(len(nearest_edges))
nearest_edges['edB2'] = np.zeros(len(nearest_edges))
nearest_edges['edB'] = np.zeros(len(nearest_edges))
nearest_edges['ndB1'] = np.zeros(len(nearest_edges))
nearest_edges['ndB2'] = np.zeros(len(nearest_edges))
nearest_edges['ndB'] = np.zeros(len(nearest_edges))


for edge in nearest_edges.index:
    idxs = stations_df['nearest_ed'] == edge;
    idxs1 = stations_df['nearest_ed1'] == edge;
    #if sum(idxs) > 0  :
    nearest_edges['ddB1'][edge] = stations_df['m_day'][idxs].mean()
    nearest_edges['edB1'][edge] = stations_df['m_eve'][idxs].mean()
    nearest_edges['ndB1'][edge] = stations_df['m_nig'][idxs].mean()
        
    #if sum(idxs1) > 0: 
    nearest_edges['ddB2'][edge] = stations_df['m_day'][idxs1].mean()
    nearest_edges['edB2'][edge] = stations_df['m_eve'][idxs1].mean()
    nearest_edges['ndB2'][edge] = stations_df['m_nig'][idxs1].mean()
    
nearest_edges['ddB'] = nearest_edges[['ddB1', 'ddB2']].mean(axis=1);
nearest_edges['edB'] = nearest_edges[['edB1', 'edB2']].mean(axis=1);
nearest_edges['ndB'] = nearest_edges[['ndB1', 'ndB2']].mean(axis=1);

nearest_edges.drop(labels=['ddB1', 'ddB2', 'edB1', 'edB2', 'ndB1', 'ndB2'], inplace=True, axis=1)

Nearest edge interpolation

In [34]:
df_to_add['ddB'] = np.empty(len(df_to_add))
df_to_add['edB'] = np.empty(len(df_to_add))
df_to_add['ndB'] = np.empty(len(df_to_add))
df_to_add['ddB'][df_to_add['ddB'].notna()] = np.nan;
df_to_add['edB'][df_to_add['edB'].notna()] = np.nan;
df_to_add['ndB'][df_to_add['ndB'].notna()] = np.nan;

for edge in df_to_add.index:
    nedge = df_to_add['n_edge'][edge]
    if nedge in nearest_edges['edges']:
        df_to_add['ddB'][edge] = nearest_edges['ddB'][nedge];
        df_to_add['edB'][edge] = nearest_edges['edB'][nedge];
        df_to_add['ndB'][edge] = nearest_edges['ndB'][nedge];

In [35]:
l1 = len(df_to_add)
l2 = 0
while l1-l2 > 0:
    l1 = df_to_add['ddB'].isna().sum()
    temp = df_to_add[df_to_add.isna()]
    for edge in temp.index:
        nedge = df_to_add['n_edge'][edge]
        if nedge in df_to_add['edges']:
            df_to_add['ddB'][edge] = df_to_add['ddB'][nedge]
            df_to_add['edB'][edge] = df_to_add['edB'][nedge]
            df_to_add['ndB'][edge] = df_to_add['ndB'][nedge]
    
    l2 = df_to_add['ddB'].isna().sum()
    
print(df_to_add['ddB'].isna().sum())
    


1043


In [25]:
# save unusable nearest edges
temp = df_to_add[df_to_add.isna()]
for edge in temp.index:
    df_to_add['nn_edge'][edge].append(df_to_add['n_edge'][edge]) 

# save filled values   
temp = df_to_add[df_to_add['ddB'].notna()]
temp.drop(labels=['n_edge', 'nn_edge'], inplace=True, axis=1)    
nearest_edges = pd.concat([df_to_add,temp], ignore_index=True)
nearest_edges.set_index(nearest_edges['edges'], inplace=True)
nearest_edges.drop_duplicates(subset='edges', keep='first', inplace=True)

# erase saved values from df_to_add
df_to_add.drop(temp.index, inplace=True)

In [26]:
df_to_add = get_close_edges(df_to_add, nearest_edges, graph)

0.09407337723424271
0.18814675446848542
0.28222013170272814
0.37629350893697083
0.4703668861712135
0.5644402634054563
0.658513640639699
0.7525870178739417
0.8466603951081844
0.940733772342427
1.03480714957667
1.1288805268109126
1.2229539040451554
1.317027281279398
1.4111006585136407
1.5051740357478833
1.5992474129821261
1.6933207902163687
1.7873941674506115
1.881467544684854
1.975540921919097
2.06961429915334
2.1636876763875823
2.257761053621825
2.351834430856068
2.4459078080903107
2.539981185324553
2.634054562558796
2.7281279397930387
2.8222013170272815
2.916274694261524
3.0103480714957667
3.1044214487300095
3.1984948259642523
3.2925682031984946
3.3866415804327374
3.4807149576669802
3.574788334901223
3.6688617121354654
3.762935089369708
3.8570084666039515
3.951081843838194
4.045155221072436
4.13922859830668
4.233301975540922
4.327375352775165
4.421448730009407
4.51552210724365
4.609595484477893
4.703668861712136
4.797742238946378
4.891815616180621
4.985888993414863
5.079962370649106
5

In [ ]:
df_to_add.drop(labels='n_edge', inplace=True, axis=1)    
nearest_edges1 = pd.concat([df_to_add,nearest_edges], ignore_index=True)
nearest_edges1.set_index(nearest_edges1['edges'], inplace=True)

In [ ]:
cnt = 0
for edge in temp.index:
    if df_to_add['n_edge'][edge] in df_to_add['edges']:
        cnt = cnt+1

In [ ]:
print(cnt)
print(len(temp))

In [ ]:
norm = plt.Normalize(0, 70)

# Creating a list of colors corresponding to the values
nearest_edges['day'] = nearest_edges['ddB'].apply(lambda x: colormap(norm(x)))
nearest_edges['eve'] = nearest_edges['ddB'].apply(lambda x: colormap(norm(x)))
nearest_edges['nig'] = nearest_edges['ddB'].apply(lambda x: colormap(norm(x)))

nearest_edges['hday'] = nearest_edges['day'].apply(lambda x: colors.to_hex(x))
nearest_edges['heve'] = nearest_edges['eve'].apply(lambda x: colors.to_hex(x))
nearest_edges['hnig'] = nearest_edges['nig'].apply(lambda x: colors.to_hex(x))

# X) Print map #

In [ ]:
import ast
graph3 = graph.edge_subgraph(nearest_edges['edges'])
m = folium.Map(location=center, zoom_start=13)

data = ox.graph_to_gdfs(graph3, nodes=False)

for (lati, long) in zip(stations_df['latitude'], stations_df['longitude']):
    folium.Marker(location=(long, lati)).add_to(m)

# Define a function to style the road features
def style_function(feature):
    id = ast.literal_eval(feature.get('id'));
    color = 'red';
    if len(id) == 3:
        color = nearest_edges['hday'][id];
    return {
        'color': color,  # Set the color of the roads (red in this example)
        'weight': 8,         # Set the line weight
        'opacity': 0.7       # Set the line opacity
    }

# Add the road network to the folium map as a GeoJSON layer with the specified style
folium.GeoJson(data, style_function=style_function).add_to(m)
m


# TESTING #

In [19]:
del df_to_add
nearest_edges.drop(labels=['ddB', 'edB', 'ndB'], inplace=True, axis=1)

In [ ]:
temp = nearest_edges[['ddB1', 'ddB2']].mean(axis=1)

In [18]:
nearest_edges.to_csv('temp.csv')

In [ ]:
nearest_edges = pd.read_csv('temp.csv')

In [ ]:
l= graph.neighbors(nearest_edges['edges'].iloc[1][1])